# Импорт библиотек #

In [1]:
import  pandas as pd
from pymystem3 import Mystem
from IPython.core.display import display
import seaborn as sns

C:\Users\Marko\AppData\Local\Temp\ipykernel_5660\1504868038.py:3: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display


## Осмотр данных ##

In [2]:
data = pd.read_csv(filepath_or_buffer='datasets/data.csv')
data.head(10)

FileNotFoundError: [Errno 2] No such file or directory: 'datasets/data.csv'

In [ ]:
data.info()

In [ ]:
sns.boxplot(data=data['children'])

In [ ]:
sns.boxplot(data=data['days_employed'] / 365)

In [ ]:
sns.boxplot(data=data['dob_years'])

In [ ]:
sns.boxplot(data=data['total_income'])

**Вывод**
1. data['days_employed']:
    - отрицательные значения/дробные значения - как интерпретировать?
2. data['education']:
    - изменить тип на 'category'
3. data['education_id']:
    - удалить
4. data['family_status']:
    - изменить тип на 'category'
5. data['family_status_id']:
    - удалить
6. data['gender']:
    - изменить тип на 'category'
7. data['income_type']:
    - изменить тип на 'category'
8. data['total_income']:
    - предложить варианты сегментации заемщиков
9. data['purpose']:
    - выделить основные сегменты
10. обнаружены аномальные данные data['children'] = 20
11. обнаружены аномальные данные data['days_employed'] - стаж более 800 лет
12. обнаружены аномальные данные data['dob_years'] - 0, 80

## Предобработка данных ##

### Обработка пропусков

In [ ]:
data[data['dob_years'] < 18]['dob_years'].unique()

In [ ]:
data[data['dob_years'] < 18] = 18

In [ ]:
print(data.isna().sum())
data = data.fillna(0)
print(data.isna().sum())

#### Замена значений data['children'] ####

In [ ]:
print(data['children'].unique())
data[data['children'] == -1] = 0
print(data['children'].unique())

**Вывод**
1. произведена замена пропущенных значений на 0 для:
    - data['days_employed'] - косвенные данные указывающие на стаж заемщика отсутствуют
      Возможные причины возникновния:
      * заемщик не указал свой стаж
      * человевеский фактор
      * потеря данных при формировании dataset
      * пропуски составляют ~10% dataset, поэтому не удаляю
    - data['total_income'] - косвенные данные указывающие на заработок заемщика отсутствуют
      Возможные причины возникновния:
      * заемщик не указал свой стаж
      * человевеский фактор
      * потеря данных при формировании dataset
      * пропуски составляют ~10% dataset, поэтому не удаляю
2. пропущенные значения удалены
3. произведена замена значения "-1" data['children'] на 0, т.к.
    - отрицательное количество детей невозможно
    - косвенные данные указывающие на количество детей отсутствуют
4. в dataset присутствует заёмщик с 20 детьми
    - данные позволяющие подтвердить указанное количество отсутствуют
    Возможные причины возникновения:
    - опечатка
    - действительно 20 детей
    Оставляю без изменений.

### Изменение типов данных ###

#### Перевод значений строковых переменных в нижний регистр ####

In [ ]:
def lower_data():
    for column in data.columns:
        if data.dtypes[column] == 'object':
            data[column] = data[column].str.strip()
            data[column] = data[column].str.lower()

lower_data()

In [ ]:
data

In [ ]:
data.info()

In [ ]:
data['days_employed'] = data['days_employed'].astype('int')
data['gender'] = data['gender'].astype('category')
data['income_type'] = data['income_type'].astype('category')
data['education'] = data['education'].astype('category')
data['family_status'] = data['family_status'].astype('category')
data.info()

**Вывод**

1. строковые значения переведены в нижний регистр
2. тип данных data['days_employed'] изменен на 'int'
3. тип данных изменен на 'category' для следующих столбцов:
    - data['gender']
    - data['income_type']
    - data['education']
    - data['family_status']


### Обработка дубликатов ###

In [ ]:
data.duplicated().sum()

#### Удаление cтолбцов data['education_id'], data['family_status_id'] ####

In [ ]:
data = data.drop(columns=['family_status_id', 'education_id'])
data.info()

**Вывод**
1. дубликаты удалены
2. удалены столбцы
    - data['family_status_id']
    - data['education_id']
   т.к. дублируют данные содержащиеся в data['education'] и data['family_status']

### Лемматизация ###

#### Лемматизация data['purpose'] ####

In [ ]:
data.isna().sum()
data = data.dropna().reset_index(drop=True)

In [ ]:
data.info()

In [ ]:
m = Mystem()
for purpose in data['purpose'].unique():
    lemma = ' '.join(m.lemmatize(purpose))
    for _ in range(len(data)):
        if purpose in data.loc[_, 'purpose']:
            data.loc[_, 'purpose'] = lemma

#### Категоризация data['purpose'] ####

In [ ]:
for lem_purpose in ['недвижимость', 'жилье', 'автомобиль', 'свадьба', 'образование']:
    for _ in range(len(data)):
        if lem_purpose in data.loc[_, 'purpose']:
            data.loc[_, 'purpose'] = lem_purpose

In [ ]:
data['purpose'] = data['purpose'].replace(to_replace='образование  \n',
                                          value='образование').\
                                  replace(to_replace='жилье',
                                          value='недвижимость')

In [ ]:
data['purpose'] = data['purpose'].astype('category')

In [ ]:
data.info()

In [ ]:
data.duplicated().sum()

**Вывод**
1. проведена лемматизация data['purpose']
2. тип данных data['purpose'] изменен на 'category'
3. список целей займа ограничен 4-я позициями

### Категоризация данных ###

#### Категоризация заёмщиков по количеству детей ####

In [ ]:
print(data['children'].describe())
print(data['children'].unique())

In [ ]:
for _ in range(len(data)):
    if data.loc[_, 'children'] == 0: data.loc[_, 'cat_children'] = 'бездетный'
    elif 1 <= data.loc[_, 'children'] <= 3: data.loc[_, 'cat_children'] = 'не более 3 детей'
    else: data.loc[_, 'cat_children'] = 'многодетный'


data['cat_children'] = data['cat_children'].astype('category')

#### Категоризация заемщиокв по уровню дохода ####

In [ ]:
print(data['total_income'].describe())

In [ ]:
for _ in range(len(data)):
    if data.loc[_, 'total_income'] <= 89062: data.loc[_, 'cat_total_income'] = 'низкий'
    elif 89062 < data.loc[_, 'total_income'] <= 135760: data.loc[_, 'cat_total_income'] = 'средний'
    elif 135760 < data.loc[_, 'total_income'] <= 195855: data.loc[_, 'cat_total_income'] = 'выше среднего'
    else: data.loc[_, 'cat_total_income'] = 'высокий'

**Вывод**
1. выполнена категоризация заемщиков по количеству детей.
   Критерии:
   - 0 детей - бездетный
   - от 1 до 3(включительно) - не более 3 детей
   - больше 3 детей - многодетный
2. дополнительная категоризация заемщиков по семейному положению не проводилась. будут использованы категории представленные в dataset-е
3. проведена категоризация заёмщиков по уровню дохода.
   Критерии:
   - до 89062 - низкий
   - от 89062 до 135760 - средний
   - от 135760 до 195855 - выше среднего
   - выше 195855 - высокий

##  Зависимость м/у наличием детей и возвратом кредита в срок ##

In [ ]:
child_pivot = data.pivot_table(values='debt', index='cat_children', aggfunc=['count', 'sum'])
child_pivot['% debt'] = child_pivot['sum'] / child_pivot['count'] * 100
display(child_pivot)

**Вывод**
- вероятность просроченных платежей по кредиту у бездетных заемщиков ниже чем у многодетных и заемщиков количество детей у которых не болле 3, на ~2%

## Зависимость между семейным положением и возвратом кредита в срок ##

In [ ]:
family_pivot = data.pivot_table(values='debt', index='family_status', aggfunc=['count', 'sum'])
family_pivot['% debt'] = family_pivot['sum'] / family_pivot['count'] * 100
display(family_pivot)

**Вывод**
- вероятность просроченных платежей по кредиту у заемщиков не состоящих в браке и состощих в гражданском браке выше в сравнении с:
    1. овдовевшими заемщиками на  ~3%
    2. заемщиками в разводе на ~3%
    3. заемщиками в браке на ~3%

## Зависимость между уровнем дохода и возвратом кредита в срок ##

In [ ]:
income_pivot = data.pivot_table(values='debt', index='cat_total_income', aggfunc=['count', 'sum'])
income_pivot['% debt'] = income_pivot['sum'] / income_pivot['count'] * 100
display(income_pivot)

**Вывод**
- вероятность просроченных платежей по кредиту у заемщиков со средним доходом и выше среднего выше в сравнении с:
    1. заемщиками с высоким доходом на ~2%
    2. заемщиками с низким доходом на ~2%

## Зависимость цели кредита и возвратом в срок ##

In [ ]:
purpose_pivot = data.pivot_table(values='debt', index='purpose', aggfunc=['count', 'sum'])
purpose_pivot['% debt'] = purpose_pivot['sum'] / purpose_pivot['count'] * 100
display(purpose_pivot)

**Вывод**
- самая низкия вероятность просроченных платежей по кредиту полученному для приобретения недвижимости
- высокая вероятность просроченных платежей для кредитов полученных на образование и приобретение автомобиля

## Вывод ##
В ходе обработки данных выявлены следующие аномалии:
    - возраст заемщика < 18
    - стаж > 57 лет
    - возраст заёмщика > 65 лет
    - количество детей у заемщика > 20
Если удалить заемщиков удовлетворяющих приведенным условиям результаты исследования могут измениться.

В ходе исследования выявлены зависимости возникновения задолженности по кредиту от:
    * целей кредита
    * уровня дохода заемщика
    * наличием детей у заемщика
    * семейного положения заемщика

1. Вероятность просроченных платежей по кредиту выше у заемщиков с детьми.
2. Вероятность просроченных платежей по кредиту выше у заемщиков не состоящих в браке или состоящих в гражданском браке.
3. Вероятность просроченных платежей по кредиту выше у заемщиков с уровнем дохода от 89062 до 195855.
4. Вероятность просроченных платежей по кредиту выше у по кредитам целью которых является образование или приобретение автомобиля.